In [1]:
import pathlib
from pathlib import Path
import os
import shutil
import sys

import torch
import librosa
import numpy as np

import random
from time import sleep

/home/nitro/anaconda3/envs/audio_proj/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [3]:
from src.utils import get_all_files, play_audio
from src.noise_augs import GaussianNoise, AddConst, ZeroElements #трансформации аудио, обёрнутые в найтивную апишку аугментаций торча

In [4]:
train_data = get_all_files("/media/nitro/hdd/Test_task/splitted_data/clean_train")
val_data = get_all_files("/media/nitro/hdd/Test_task/splitted_data/clean_val")
test_data = get_all_files("/media/nitro/hdd/Test_task/splitted_data/clean_testset_wav")

# Конвертация в Stft 

In [11]:
random_idx = random.randint(0,len(train_data))
print(random_idx)
waveform, sample_rate = librosa.load(train_data[random_idx]) # Возьмём случайный пример

3740


In [12]:
stft_wave = librosa.stft(waveform, n_fft=2048) #конвертируем в stft

In [ ]:
composed = torch.nn.Sequential(GaussianNoise(stft_wave.shape),                                                              
                               AddConst(constant=32, position=8), 
                              ZeroElements(zero_over_slice=(16,32)))

In [ ]:
transformed_sample = composed(stft_wave) # применяем зашумление

In [ ]:
inverse_stft = librosa.istft(transformed_sample, n_fft=2048) # возвращаем в обычный вид из stft

In [ ]:
print("Звук до зашумления")
play_audio(waveform, sample_rate)

sleep(0.3)

print("Звук после зашумления")
play_audio(inverse_stft, sample_rate) 

### Для того, чтобы понять какое оптимальное значение нужно выбрать для паддинга посторим гистограмму

In [ ]:
from tqdm import tqdm
length = []
for file in tqdm(range(len(train_data))):
    waveform, sample_rate = librosa.load(train_data[file])
    y_8k = librosa.resample(waveform, orig_sr=sample_rate, target_sr=16000)
    length.append(y_8k.shape)

In [ ]:
np.histogram(length)

In [ ]:
import seaborn as sns

sns.set_style('darkgrid')
sns.distplot(length)

По гистограмме видно, что оптимальное значение приблизительно -- 65000. Паддинг использован, чтобы не потерять много информации и не забивать ноликами массив до наибольшего размера в датасете, что привело бы к неоптимальному использованию места